# Hiding Information in AI Explanation _ Project Code

In [ ]:
## How to run the code


In [ ]:
#!git clone https://github.com/pankessel/adv_explanation_ref.git

In [2]:
 pip install -r requirements.txt 

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.


In [3]:
folder_path = "C:/users/julien/rai_project_LOCAL/adv_explanation_ref/"

In [6]:
import os
import shutil
import subprocess
import requests
from io import BytesIO

import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import torch
import torch.nn.functional as F
from torchvision import models, transforms
from torchvision.models import resnet50
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision.transforms.functional import to_pil_image
from torchcam.methods import GradCAM

from pathlib import Path
import sys
sys.path.append(str(Path().resolve() / "src"))

from nn.utils import get_expl, load_image
from nn.enums import ExplainingMethod
from nn.networks import ExplainableNet
import torchvision.models as models

imagenet_classes = requests.get("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt").text.splitlines()

model = models.resnet34(pretrained=True)
model.eval()

model_gradcam = models.resnet34(pretrained=True)
model_gradcam.eval()
cam_extractor = GradCAM(model_gradcam, target_layer=model_gradcam.layer4[-1])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



def preprocess_image(input_image, output_path):
    
    image_rgb = input_image.convert("RGB")
    image_rgb.save(output_path, format="PNG")
    
    return Image.open(output_path)


    
def classify(image):
    
    input_tensor = transform(image).unsqueeze(0)  # [1, 3, 224, 224]
    
    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        top_prob, top_class = probs.topk(1, dim=1)

    class_name = imagenet_classes[top_class.item()]
    probability = top_prob.item()
    return class_name, probability


def image_from_text(text, width, height):
    image_size = width, height
    font_path = "C:/Windows/Fonts/georgia.ttf"
    max_font_size = 500

    def find_best_font_size(text, image_size, font_path, max_size=500):
        for font_size in reversed(range(10, max_size, 2)):
            try:
                font = ImageFont.truetype(font_path, font_size)
            except:
                continue
            dummy_img = Image.new('L', image_size)
            draw = ImageDraw.Draw(dummy_img)
            bbox = draw.textbbox((0, 0), text, font=font)
            width, height = bbox[2] - bbox[0], bbox[3] - bbox[1]
            if width <= image_size[0] and height <= image_size[1]:
                return font_size
        return 10

    font_size = find_best_font_size(text, image_size, font_path, max_font_size)
    font = ImageFont.truetype(font_path, font_size)

    img = Image.new('L', image_size, color=0)
    draw = ImageDraw.Draw(img)
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]
    text_position = ((image_size[0] - text_width) // 2, (image_size[1] - text_height) // 2)
    draw.text(text_position, text, fill=255, font=font)

    return img


#gradcam method
def saliency_map_gen(input_image, model=model_gradcam, device="cuda"):
    model = model.to(device)
    
    
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    
    if isinstance(input_image, Image.Image):
        x = preprocess(input_image).unsqueeze(0).to(device)
    elif isinstance(input_image, torch.Tensor):
        x = input_image.clone().detach().to(device)
        if x.dim() == 3:
            x = x.unsqueeze(0)
    else:
        raise ValueError("Unsupported input type for saliency_map_gen")

    x.requires_grad = True

    output = model(x)
    class_idx = output.argmax(dim=1).item()
    score = output[0, class_idx]
    score.backward()

    saliency = x.grad * x
    saliency = saliency.abs().sum(dim=1, keepdim=True)
    saliency = saliency - saliency.min()
    saliency = saliency / (saliency.sum() + 1e-8)
    saliency = F.interpolate(saliency, size=(224, 224), mode='bilinear', align_corners=False)

    return saliency.detach()



#lrp method
def get_lrp_map(image_path, model=None, device=None, folder_path=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    #ImageNet normalisation
    data_mean = [0.485, 0.456, 0.406]
    data_std = [0.229, 0.224, 0.225]

    full_path = Path(os.path.join(folder_path, image_path)) if folder_path else Path(image_path)
    x = load_image(data_mean, data_std, device, full_path)

    if model is None:
        base_model = models.vgg16(pretrained=True)
        model = ExplainableNet(model=base_model).eval().to(device)

    expl_map, _, _ = get_expl(model, x, ExplainingMethod.lrp)

    return expl_map.squeeze(0).detach()



def run(input_image, text_to_encode):
    
    #cleaning the temporary files
    temp_folder_path = os.path.join(folder_path, "temp/")
    
    if os.path.exists(temp_folder_path):
        shutil.rmtree(temp_folder_path)
        
    os.makedirs(temp_folder_path)


    #taking care of the original input image
    temp_path_input_image = os.path.join(folder_path, "temp/temp_input_image.png")
    preprocessed_input_image = preprocess_image(input_image, temp_path_input_image)

    
    #taking care of the text
    width,height = input_image.size
    text_image = image_from_text(text_to_encode, width, height)
    temp_path_text_image = os.path.join(folder_path, "temp/temp_text_image.png")
    preprocessed_text_image = preprocess_image(text_image, temp_path_text_image)

    #checking the original classification and its probability
    print("checking the original class and probability...")
    original_class, original_probability = classify(preprocessed_input_image)

    python_file_folder_path = os.path.join(folder_path, "src/")
    print("running the script...")
    subprocess.run(["python", "run_attack.py", "--cuda", "--img", temp_path_input_image, "--target_img", temp_path_text_image], cwd=python_file_folder_path)

    
    outputs_path = os.path.join(folder_path, "output/")

    print("copying the images...")
    images = {}
    for i in ["manipulated_image.png", "original_expl.png", "target_expl.png", "manipulated_expl.png"]:
        shutil.copy(os.path.join(folder_path, f"output/{i}"), os.path.join(folder_path, f"temp/temp_{i}"))
        key = i.replace(".png", "")
        images[key] = Image.open(os.path.join(folder_path, f"temp/temp_{i}"))
  
    original_expl = images["original_expl"]
    target_expl = images["target_expl"]
    manipulated_expl = images["manipulated_expl"]
    manipulated_image = images["manipulated_image"]
 
  
    new_class, new_probability = classify(manipulated_image)

    return original_class, original_probability, original_expl, target_expl, manipulated_expl, manipulated_image, new_class, new_probability

C:\Users\Julien\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


no display found. Using non-interactive Agg backend


C:\Users\Julien\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Julien\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
demo = gr.Interface(fn=run,
                       inputs=[gr.Image(type="pil"), "text"],
                       outputs=["text", "text", gr.Image(type="pil", label="Original saliency"), gr.Image(type="pil", label="Text saliency"), gr.Image(type="pil", label="Manipulated saliency"), gr.Image(type="pil", label="Reconstructed image"), "text", "text"],
                       title="Hiding information in AI explanations",
                       description="Here you can upload a picture of your choice and choose the text that should be displayed as a ")
demo.launch(debug=True)

In [1]:
#manipulated image explanation can be tested here

image_path = os.path.join(folder_path, "temp/temp_manipulated_image.png")

lrp_map = get_lrp_map(image_path, device="cuda")
plt.imshow(lrp_map.cpu().numpy(), cmap="hot")
plt.axis("off")
plt.show()



NameError: name 'os' is not defined

In [ ]:
#!python C:\users\julien\rai_project_LOCAL\adv_explanation_ref\src\run_attack.py --cuda